In [1]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import NCF  # pure data, 
from libreco.evaluation import evaluate
from libreco.algorithms import WideDeep

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
data = pd.read_csv("../assets/data/converted.csv", sep=";")
data['rating'] += 10
print(data.head())

   user  restaurant  rating
0     1           1    2.18
1     1           2   18.79
2     1           3    0.34
3     1           4    1.84
4     1           5    2.48


In [3]:
data.columns = ['user', 'item', 'label']
train_data, eval_data, test_data = random_split(data, multi_ratios=[0.9, 0.1, 0.1])

In [4]:
train_data, data_info= DatasetPure.build_trainset(train_data)
eval_data = DatasetPure.build_evalset(eval_data)
test_data = DatasetPure.build_testset(test_data)

In [ ]:
model = WideDeep(
    task="rating",
    data_info=data_info,
    embed_size=16,
    n_epochs=4,
    loss_type="cross_entropy",
    lr={"wide": 0.05, "deep": 7e-4},
    batch_size=2048,
    use_bn=True,
    hidden_units=(128, 64, 32),
)

In [6]:
model.fit(
    train_data,
    neg_sampling=False,  # perform negative sampling on training and eval data
    verbose=2,
    shuffle=True,
    eval_data=eval_data,
    metrics=["mae"],
)

Training start time: 2024-11-21 16:34:23
Instructions for updating:
Colocations handled automatically by placer.


C:\Users\Thomas Chourret\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\libreco\layers\dense.py:31: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
2024-11-21 16:34:23,424 - WARNING - From C:\Users\Thomas Chourret\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\layers\normalization\batch_normalization.py:581: _colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


total params: 1,264,935 | embedding params: 1,250,117 | network params: 14,818


C:\Users\Thomas Chourret\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\libreco\layers\dense.py:39: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  net = tf.layers.batch_normalization(net, training=is_training)
train: 100%|██████████| 3274/3274 [00:26<00:00, 124.74it/s]


Epoch 1 elapsed: 26.261s
	 train_loss: 22.5727


...

Epoch 5 elapsed: 29.222s
	 train_loss: 12.9138


eval_pointwise: 100%|██████████| 46/46 [00:00<00:00, 117.59it/s]


	 eval mae: 3.2722


train:  13%|█▎        | 435/3274 [00:04<00:27, 101.57it/s]


KeyboardInterrupt: 

In [ ]:
template_file = "../assets/data/template.csv"
output_file = "../assets/data/output_neural_network.csv"

with open(template_file, "r") as f:
	for line in f:
		user, item, x = line.strip().split(";")
		user = int(user.replace("User", ""))
		item = int(item.replace("Restaurant", ""))
		prediction = model.predict(user, item) - 10
		print(f"User {user} will rate item {item} with {prediction}")
		with open(output_file, "a") as o:
			o.write(f"User{user};Restaurant{item};{prediction}\n")

User 25287 will rate item 31 with -1.6632518768310547
User 45110 will rate item 50 with 3.8613739013671875
User 53493 will rate item 19 with 2.5809574127197266
User 16630 will rate item 10 with 0.39593029022216797
User 48335 will rate item 8 with 0.34954071044921875
User 9619 will rate item 90 with -0.6248874664306641
User 67625 will rate item 48 with -3.6843557357788086
User 27976 will rate item 57 with -0.2682056427001953
User 10527 will rate item 17 with 1.4777441024780273
User 26355 will rate item 2 with -1.1646814346313477
User 12571 will rate item 18 with -1.6571359634399414
User 30307 will rate item 61 with 3.5644702911376953
User 17802 will rate item 71 with 2.8962745666503906
User 18767 will rate item 12 with 2.858173370361328
User 36277 will rate item 69 with 4.879100799560547
User 34746 will rate item 41 with 4.324568748474121
User 6985 will rate item 20 with -2.094254493713379
User 14851 will rate item 15 with -4.333962440490723
User 32797 will rate item 42 with 2.387218475